In [1]:
import time
import pandas as pd
import numpy as np
import math

# internal tools
from dsgtools.reporting import make_format
from dsgtools.reporting import TableWriter
from dsgtools.reporting import freq
from dsgtools.reporting import bivariate

In [2]:
## new fix for data import
import os
os.environ["HTTP_PROXY"] = ""
os.environ["HTTPS_PROXY"] = ""
os.environ["NO_PROXY"] = ""

In [3]:
## Input Data
chase = pd.read_adls("Analytics/RnD Projects/Product RnD/Business/LexisNexis 9999 (SBFE Inquiry POC)/Performance/Fraud Sample/Chase_11449/" + "chase_stacked_final.parquet")
print(chase.shape)

tmobile = pd.read_adls("Analytics/RnD Projects/Product RnD/Business/LexisNexis 9999 (SBFE Inquiry POC)/Performance/Fraud Sample/tmobile_10823/" + "tmbile_file_to_use.parquet")
print(tmobile.shape)

stripe = pd.read_adls("Analytics/RnD Projects/Product RnD/Business/LexisNexis 9999 (SBFE Inquiry POC)/Performance/Fraud Sample/stripe_11363/" + "stripe_file_to_use.parquet")
print(stripe.shape)

fleector = pd.read_adls("Analytics/RnD Projects/Product RnD/Business/LexisNexis 9999 (SBFE Inquiry POC)/Performance/Fraud Sample/fleector_11124/" + "fleector_sample_to_use.parquet")
print(fleector.shape)

(640958, 36)
(114460, 39)
(75000, 22)
(83704, 92)


In [4]:
## keep seleid, full address, phone, taxid, performance flag >> business information >> might need to dedup

#### SBFE Inquiry

In [5]:
path = "Analytics/RnD Projects/Product RnD/Business/LexisNexis 9999 (SBFE Inquiry POC)/Data Modeling Clean/sbfe_seleid_append_deduped_cleaned_0501.parquet"
sbfe = pd.read_adls(path, reader = pd.read_parquet)
print(sbfe.shape)

Bad pipe message: %s [b'\x1b\xa3\xf6\xe2\x89\x85\x0c\xafS\xf7\x89\xb8\xa2\xef3.s\xd6 \x88\xbd\xde\xdb\x98\xed\x1e\x87K\xf6\x8e\xaaWxk\x8a~y\xb9<\x1dr\x95i\x8d\xd9\xfe\x01#tQ\x9a\x00\x08\x13\x02\x13\x03\x13\x01\x00\xff\x01\x00\x00\x8f\x00\x00\x00\x0e\x00\x0c\x00\x00\t127.0.0.1\x00\x0b\x00\x04\x03\x00\x01\x02\x00\n\x00\x0c\x00\n\x00\x1d\x00\x17\x00\x1e\x00\x19\x00\x18\x00#\x00\x00\x00\x16\x00\x00\x00\x17\x00\x00\x00\r\x00\x1e\x00\x1c\x04\x03\x05\x03\x06\x03\x08\x07\x08\x08\x08\t\x08', b'\x0b\x08\x04\x08\x05\x08\x06\x04\x01']
Bad pipe message: %s [b'\x04M\x98N`l\xe0\x18\x8dl_*~\xba\xf1\xbd&8\x00\x00>\xc0\x14\xc0\n\x009\x008\x007\x006\xc0\x0f\xc0\x05\x005\xc0\x13\xc0\t\x003\x002\x00', b'0\xc0\x0e\xc0\x04\x00/\x00\x9a\x00\x99\x00\x98\x00\x97\x00\x96\x00\x07\xc0\x11\xc0\x07\xc0\x0c\xc0\x02\x00\x05\x00\x04\x00\xff\x02\x01\x00\x00C\x00\x00\x00\x0e\x00\x0c\x00\x00\t1']
Bad pipe message: %s [b"\xbaS\xa0\xeayu({t;\xccz\xa2\x19K\xf2\xb1\x08\x00\x00|\xc0,\xc00\x00\xa3\x00\x9f\xcc\xa9\xcc\xa8\xcc\xa

(79429086, 20)


In [6]:
sbfe["full_addr"] = sbfe["addr"] + np.where(sbfe["city"] != "", ", " + sbfe["city"], "") + \
    np.where(sbfe["state"] != "", ", " + sbfe["state"], "") + np.where(sbfe["zip"] != "", ", " + sbfe["zip"].str.slice(0, 5), "")
sbfe["full_addr_clean"] = sbfe['full_addr'].str.replace('.', '')
sbfe["full_addr_clean"] = sbfe['full_addr_clean'].str.replace(',', '')
sbfe["full_addr_clean"] = sbfe['full_addr_clean'].str.replace('\W\s', '')
sbfe["full_addr_clean"] = sbfe["full_addr_clean"].str.lower()

/tmp/ipykernel_25390/3858317675.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  sbfe["full_addr_clean"] = sbfe['full_addr'].str.replace('.', '')
/tmp/ipykernel_25390/3858317675.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  sbfe["full_addr_clean"] = sbfe['full_addr_clean'].str.replace('\W\s', '')


In [7]:
sbfe.columns

Index(['accountnumber_x', 'companyname', 'alternatecompanyname_x', 'addr',
       'city', 'state', 'zip', 'businessphone', 'taxidnumber', 'historydate_x',
       'sufficient_input', 'accountnumber_y', 'alternatecompanyname_y',
       'historydate_y', 'powid', 'proxid', 'seleid', 'orgid', 'ultid',
       'overallweight', 'full_addr', 'full_addr_clean'],
      dtype='object')

In [8]:
sbfe = sbfe[["seleid", "historydate_x", "full_addr_clean", 'businessphone', 'taxidnumber']]
sbfe.columns = ["seleid_inquiry_sbfe", "inquiry_date", "full_addr_clean", 'businessphone', 'taxidnumber']

In [9]:
sbfe["inquiry_date"] = sbfe["inquiry_date"].astype(int)
sbfe["inquiry_date_fmt"] = pd.to_datetime(sbfe["inquiry_date"], format='%Y%m%d')

In [10]:
sbfe["seleid_inquiry_sbfe"] = sbfe["seleid_inquiry_sbfe"].astype(float)

In [11]:
sbfe["full_addr_clean"] = sbfe["full_addr_clean"].str.lower()

In [12]:
sbfe.head()

,seleid_inquiry_sbfe,inquiry_date,full_addr_clean,businessphone,taxidnumber,inquiry_date_fmt
0,1.336156e+11,20210701,115 golf course rd ste e logan ut 84321,4355353654,,2021-07-01
1,1.330122e+09,20210701,3200 palm tree dr lithonia ga 30038,7708857033,,2021-07-01
2,9.064136e+08,20210701,1476 highway 159 e bellville tx 77418,9798653142,,2021-07-01
5,6.136772e+07,20210701,492 koller st san francisco ca 94110,6505550000,,2021-07-01
6,1.371452e+11,20210701,107 lassiter lane bellville tx 77418,9798859397,,2021-07-01


#### Stripe 11363

In [15]:
stripe = stripe.fillna("")
stripe = stripe.drop_duplicates(subset = ['companyname', 'alternatecompanyname', 'bus_streetaddress1', 
                                          'bus_streetaddress2', 'bus_city', 'bus_state', 'bus_zip', 'bus_phone', 'bus_website'], ignore_index = True) 
print(stripe.shape)
print(stripe.uniqueid.is_unique)

(74443, 22)
True


In [16]:
# Compile Full Address: np.where(stripe["bus_streetaddress2"] != "", ", " + stripe["bus_streetaddress2"], "") + 
stripe["full_addr"] = stripe["bus_streetaddress1"] + \
    np.where(stripe["bus_city"] != "", ", " + stripe["bus_city"], "") + \
    np.where(stripe["bus_state"] != "", ", " + stripe["bus_state"], "") + np.where(stripe["bus_zip"] != "", ", " + stripe["bus_zip"].str.slice(0, 5), "")
stripe["full_addr_clean"] = stripe['full_addr'].str.replace('.', '')
stripe["full_addr_clean"] = stripe['full_addr_clean'].str.replace(',', '')
stripe["full_addr_clean"] = stripe['full_addr_clean'].str.replace('\W\s', '')
stripe["full_addr_clean"] = stripe["full_addr_clean"].str.lower()

/tmp/ipykernel_24833/2995735239.py:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  stripe["full_addr_clean"] = stripe['full_addr'].str.replace('.', '')
/tmp/ipykernel_24833/2995735239.py:7: FutureWarning: The default value of regex will change from True to False in a future version.
  stripe["full_addr_clean"] = stripe['full_addr_clean'].str.replace('\W\s', '')


In [17]:
freq(stripe.historydate.str.slice(0, 7))

,Count,Pct,Cuml Count,Cuml Pct
historydate,,,,
2022-01,24426.0,0.328117,24426.0,0.328117
2022-02,24209.0,0.325202,48635.0,0.653319
2022-03,25808.0,0.346681,74443.0,1.000000


In [18]:
freq(stripe.performance_indicator)

,Count,Pct,Cuml Count,Cuml Pct
performance_indicator,,,,
False,73807.0,0.991457,73807.0,0.991457
True,636.0,0.008543,74443.0,1.000000


In [19]:
stripe["bad"] = np.where(stripe.performance_indicator == "True", 1, 0)

In [20]:
stripe[stripe["full_addr_clean"]!=""].shape

(74438, 25)

In [22]:
stripe = stripe[stripe["full_addr_clean"]!=""]
print(stripe.shape)

(74438, 25)


In [23]:
stripe["credit_flag_cutoff_date"] = stripe["historydate"].str.slice(0, 10)
stripe["credit_flag_cutoff_date_fmt"] = pd.to_datetime(stripe["credit_flag_cutoff_date"], format='%Y-%m-%d')

In [24]:
sbfe_for_stripe = sbfe.merge(stripe, on = "full_addr_clean")
print(sbfe_for_stripe.shape)
sbfe_for_stripe = sbfe_for_stripe[sbfe_for_stripe["inquiry_date_fmt"] < sbfe_for_stripe["credit_flag_cutoff_date_fmt"]]
print(sbfe_for_stripe.shape)

(410374, 32)
(250294, 32)


In [25]:
sbfe_for_stripe.columns

Index(['seleid_inquiry_sbfe', 'inquiry_date', 'full_addr_clean',
       'businessphone', 'taxidnumber', 'inquiry_date_fmt', 'Unnamed: 0',
       'uniqueid', 'historydate', 'companyname', 'alternatecompanyname',
       'bus_streetaddress1', 'bus_streetaddress2', 'bus_city', 'bus_state',
       'bus_zip', 'bus_phone', 'bus_website', 'firstname', 'lastname',
       'streetaddress1', 'streetaddress2', 'city', 'state', 'zip', 'email',
       'ip_address', 'performance_indicator', 'full_addr', 'bad',
       'credit_flag_cutoff_date', 'credit_flag_cutoff_date_fmt'],
      dtype='object')

In [38]:
temp = sbfe_for_stripe[(sbfe_for_stripe["credit_flag_cutoff_date_fmt"] - sbfe_for_stripe["inquiry_date_fmt"])/np.timedelta64(1, 'M') <= 6].copy()
count_unique_06 = temp.groupby(by = ['uniqueid'])["businessphone"].nunique().reset_index()
print(count_unique_06.shape)

temp = sbfe_for_stripe[(sbfe_for_stripe["credit_flag_cutoff_date_fmt"] - sbfe_for_stripe["inquiry_date_fmt"])/np.timedelta64(1, 'M') <= 3].copy()
count_unique_03 = temp.groupby(by = ['uniqueid'])["businessphone"].nunique().reset_index()
print(count_unique_03.shape)

temp = sbfe_for_stripe[(sbfe_for_stripe["credit_flag_cutoff_date_fmt"] - sbfe_for_stripe["inquiry_date_fmt"])/np.timedelta64(1, 'M') <= 1].copy()
count_unique_01 = temp.groupby(by = ['uniqueid'])["businessphone"].nunique().reset_index()
print(count_unique_01.shape)

temp = sbfe_for_stripe[(sbfe_for_stripe["credit_flag_cutoff_date_fmt"] - sbfe_for_stripe["inquiry_date_fmt"])/np.timedelta64(1, 'W') <= 1].copy()
count_unique_1w = temp.groupby(by = ['uniqueid'])["businessphone"].nunique().reset_index()
print(count_unique_1w.shape)

(10006, 2)
(8574, 2)
(6801, 2)
(3118, 2)


In [37]:
count_unique_06.columns = ['uniqueid', "phn_count_unique_06"]
count_unique_03.columns = ['uniqueid', "phn_count_unique_03"]
count_unique_01.columns = ['uniqueid', "phn_count_unique_01"]
count_unique_1w.columns = ['uniqueid', "phn_count_unique_1w"]

stripe = stripe.merge(count_unique_06, on = "uniqueid", how = "left")
print(stripe.shape)
stripe = stripe.merge(count_unique_03, on = "uniqueid", how = "left")
print(stripe.shape)
stripe = stripe.merge(count_unique_01, on = "uniqueid", how = "left")
print(stripe.shape)
stripe = stripe.merge(count_unique_1w, on = "uniqueid", how = "left")
print(stripe.shape)

(74438, 28)
(74438, 29)
(74438, 30)
(74438, 31)


In [39]:
temp = sbfe_for_stripe[(sbfe_for_stripe["credit_flag_cutoff_date_fmt"] - sbfe_for_stripe["inquiry_date_fmt"])/np.timedelta64(1, 'M') <= 6].copy()
count_unique_06 = temp.groupby(by = ['uniqueid'])["taxidnumber"].nunique().reset_index()
print(count_unique_06.shape)

temp = sbfe_for_stripe[(sbfe_for_stripe["credit_flag_cutoff_date_fmt"] - sbfe_for_stripe["inquiry_date_fmt"])/np.timedelta64(1, 'M') <= 3].copy()
count_unique_03 = temp.groupby(by = ['uniqueid'])["taxidnumber"].nunique().reset_index()
print(count_unique_03.shape)

temp = sbfe_for_stripe[(sbfe_for_stripe["credit_flag_cutoff_date_fmt"] - sbfe_for_stripe["inquiry_date_fmt"])/np.timedelta64(1, 'M') <= 1].copy()
count_unique_01 = temp.groupby(by = ['uniqueid'])["taxidnumber"].nunique().reset_index()
print(count_unique_01.shape)

temp = sbfe_for_stripe[(sbfe_for_stripe["credit_flag_cutoff_date_fmt"] - sbfe_for_stripe["inquiry_date_fmt"])/np.timedelta64(1, 'W') <= 1].copy()
count_unique_1w = temp.groupby(by = ['uniqueid'])["taxidnumber"].nunique().reset_index()
print(count_unique_1w.shape)

(10006, 2)
(8574, 2)
(6801, 2)
(3118, 2)


In [40]:
count_unique_06.columns = ['uniqueid', "taxid_count_unique_06"]
count_unique_03.columns = ['uniqueid', "taxid_count_unique_03"]
count_unique_01.columns = ['uniqueid', "taxid_count_unique_01"]
count_unique_1w.columns = ['uniqueid', "taxid_count_unique_1w"]

stripe = stripe.merge(count_unique_06, on = "uniqueid", how = "left")
print(stripe.shape)
stripe = stripe.merge(count_unique_03, on = "uniqueid", how = "left")
print(stripe.shape)
stripe = stripe.merge(count_unique_01, on = "uniqueid", how = "left")
print(stripe.shape)
stripe = stripe.merge(count_unique_1w, on = "uniqueid", how = "left")
print(stripe.shape)

(74438, 32)
(74438, 33)
(74438, 34)
(74438, 35)


In [41]:
stripe['phn_count_unique_06'] = np.where(stripe["phn_count_unique_06"].isnull(), 0,   stripe["phn_count_unique_06"])
stripe['phn_count_unique_03'] = np.where(stripe["phn_count_unique_03"].isnull(), 0,   stripe["phn_count_unique_03"])
stripe['phn_count_unique_01'] = np.where(stripe["phn_count_unique_01"].isnull(), 0,   stripe["phn_count_unique_01"])
stripe['phn_count_unique_1w'] = np.where(stripe["phn_count_unique_1w"].isnull(), 0,   stripe["phn_count_unique_1w"])

stripe['taxid_count_unique_06'] = np.where(stripe["taxid_count_unique_06"].isnull(), 0,   stripe["taxid_count_unique_06"])
stripe['taxid_count_unique_03'] = np.where(stripe["taxid_count_unique_03"].isnull(), 0,   stripe["taxid_count_unique_03"])
stripe['taxid_count_unique_01'] = np.where(stripe["taxid_count_unique_01"].isnull(), 0,   stripe["taxid_count_unique_01"])
stripe['taxid_count_unique_1w'] = np.where(stripe["taxid_count_unique_1w"].isnull(), 0,   stripe["taxid_count_unique_1w"])

In [42]:
fmt = make_format(cuts = [-np.inf, 0, 1, 2, 3, 4, 5, 10, 20, 30, 40, 50, np.inf], exceptions = [-1])
result = dict()
keep = ['phn_count_unique_06', 'phn_count_unique_03', 'phn_count_unique_01', 'phn_count_unique_1w', 'taxid_count_unique_06', 'taxid_count_unique_03', 'taxid_count_unique_01', 'taxid_count_unique_1w']
for i, a in enumerate(keep):
    result[i] = bivariate(stripe[a], stripe["bad"], format = fmt)

wb = TableWriter(filename = "./_temp/stripe.xlsx", options={'nan_inf_to_errors': True}, overwrite = True)
for k in result.keys():
    wb.write_table(
        result[k],
        sheetname="biv",
        conditional_fmt_cols=[3],
    )
wb.close()

#### TMobile 10823

In [13]:
tmobile["credit_flag_cutoff_date"] = tmobile.appdatetime_gmt.str.slice(0, 9)
tmobile["credit_flag_cutoff_date_fmt"] = pd.to_datetime(tmobile["credit_flag_cutoff_date"], format='%d%b%Y')

In [14]:
freq(tmobile["credit_flag_cutoff_date_fmt"].astype(str).str.slice(0, 7))

,Count,Pct,Cuml Count,Cuml Pct
credit_flag_cutoff_date_fmt,,,,
2022-01,20510.0,0.179189,20510.0,0.179189
2022-02,22452.0,0.196156,42962.0,0.375345
2022-03,22846.0,0.199598,65808.0,0.574943
2022-04,14973.0,0.130814,80781.0,0.705757
2022-05,16271.0,0.142154,97052.0,0.847912
2022-06,17408.0,0.152088,114460.0,1.000000


In [17]:
freq(tmobile.src)

,Count,Pct,Cuml Count,Cuml Pct
src,,,,
march,33679.0,0.294243,33679.0,0.294243
retro,80781.0,0.705757,114460.0,1.000000


In [18]:
freq(tmobile.fraud_ind)

,Count,Pct,Cuml Count,Cuml Pct
fraud_ind,,,,
0,58655.0,0.512450,58655.0,0.512450
1,3762.0,0.032867,62417.0,0.545317
Missing,52043.0,0.454683,114460.0,1.000000


In [19]:
tmobile = tmobile[tmobile["fraud_ind"].notnull()]
print(tmobile.shape)

(62417, 41)


In [20]:
freq(tmobile.src)

,Count,Pct,Cuml Count,Cuml Pct
src,,,,
march,21914.0,0.35109,21914.0,0.35109
retro,40503.0,0.64891,62417.0,1.00000


In [21]:
tmobile["credit_flag_cutoff_date"] = tmobile["appdatetime_gmt"].str.slice(0, 9)
tmobile["credit_flag_cutoff_date_fmt"] = pd.to_datetime(tmobile["credit_flag_cutoff_date"], format='%d%b%Y')

In [22]:
tmobile.columns

Index(['crid_encr', 'appdatetime_gmt', 'businessname', 'business_contactphone',
       'business_addressline1', 'business_addressline2',
       'business_addresscity', 'business_addressstate',
       'business_addresspostalcode', 'federaltaxid', 'contactfirstname',
       'contactmiddlename', 'contactlastname', 'contact_contactphone',
       'contact_addressline1', 'contact_addressline2', 'contact_addresscity',
       'contact_addressstate', 'contact_addresspostalcode', 'contact_dob',
       'contact_idnum', 'contact_idtype', 'contact_idstate', 'contact_email',
       'application_channel', 'fraud_ind', 'fraud_tp_ind', 'fraud_fp_ind',
       'fpd_ind', 'any_wo_ind', 'activationflag', 'seq', 'account', 'date',
       'src', 'contact_dob_year', 'contact_dob_month', 'contact_dob_day',
       'contact_driverlicensenumber', 'credit_flag_cutoff_date',
       'credit_flag_cutoff_date_fmt'],
      dtype='object')

In [26]:
tmobile['abbrev_state'] = tmobile['business_addressstate'].map(statename_to_abbr)
print(tmobile['abbrev_state'].isnull().sum())
print(tmobile['business_addressstate'].isnull().sum())

3
3


In [27]:
# freq("business_addressstate", df=tmobile[tmobile['business_addressstate'].notnull() & tmobile['abbrev_state'].isnull()])
## ADD abbreviation to mapping dictionary

In [28]:
# Compile Full Address: np.where(stripe["bus_streetaddress2"] != "", ", " + stripe["bus_streetaddress2"], "") + 
tmobile["full_addr"] = tmobile["business_addressline1"] + \
    np.where(tmobile["business_addresscity"] != "", ", " + tmobile["business_addresscity"], "") + \
    np.where(tmobile["abbrev_state"] != "", ", " + tmobile["abbrev_state"], "") + \
        np.where(tmobile["business_addresspostalcode"] != "", ", " + tmobile["business_addresspostalcode"].str.slice(0, 5), "")
tmobile["full_addr_clean"] = tmobile['full_addr'].str.replace('.', '')
tmobile["full_addr_clean"] = tmobile['full_addr_clean'].str.replace(',', '')
tmobile["full_addr_clean"] = tmobile['full_addr_clean'].str.replace('\W\s', '')
tmobile["full_addr_clean"] = tmobile["full_addr_clean"].str.lower()

/tmp/ipykernel_25390/3611770108.py:6: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  tmobile["full_addr_clean"] = tmobile['full_addr'].str.replace('.', '')
/tmp/ipykernel_25390/3611770108.py:8: FutureWarning: The default value of regex will change from True to False in a future version.
  tmobile["full_addr_clean"] = tmobile['full_addr_clean'].str.replace('\W\s', '')


In [29]:
tmobile[tmobile["full_addr_clean"].notnull()].shape

(62414, 44)

In [30]:
tmobile = tmobile[tmobile["full_addr_clean"].notnull()]

In [32]:
tmobile.shape

(62414, 44)

In [33]:
sbfe_for_tmobile = sbfe.merge(tmobile, on = "full_addr_clean")
print(sbfe_for_tmobile.shape)
sbfe_for_tmobile = sbfe_for_tmobile[sbfe_for_tmobile["inquiry_date_fmt"] < sbfe_for_tmobile["credit_flag_cutoff_date_fmt"]]
print(sbfe_for_tmobile.shape)

(274727, 49)
(201828, 49)


In [34]:
temp = sbfe_for_tmobile[(sbfe_for_tmobile["credit_flag_cutoff_date_fmt"] - sbfe_for_tmobile["inquiry_date_fmt"])/np.timedelta64(1, 'M') <= 6].copy()
count_unique_06 = temp.groupby(by = ['crid_encr'])["businessphone"].nunique().reset_index()
print(count_unique_06.shape)

temp = sbfe_for_tmobile[(sbfe_for_tmobile["credit_flag_cutoff_date_fmt"] - sbfe_for_tmobile["inquiry_date_fmt"])/np.timedelta64(1, 'M') <= 3].copy()
count_unique_03 = temp.groupby(by = ['crid_encr'])["businessphone"].nunique().reset_index()
print(count_unique_03.shape)

temp = sbfe_for_tmobile[(sbfe_for_tmobile["credit_flag_cutoff_date_fmt"] - sbfe_for_tmobile["inquiry_date_fmt"])/np.timedelta64(1, 'M') <= 1].copy()
count_unique_01 = temp.groupby(by = ['crid_encr'])["businessphone"].nunique().reset_index()
print(count_unique_01.shape)

temp = sbfe_for_tmobile[(sbfe_for_tmobile["credit_flag_cutoff_date_fmt"] - sbfe_for_tmobile["inquiry_date_fmt"])/np.timedelta64(1, 'W') <= 1].copy()
count_unique_1w = temp.groupby(by = ['crid_encr'])["businessphone"].nunique().reset_index()
print(count_unique_1w.shape)

(12647, 2)
(9429, 2)
(5326, 2)
(2031, 2)


In [35]:
count_unique_06.columns = ['crid_encr', "phn_count_unique_06"]
count_unique_03.columns = ['crid_encr', "phn_count_unique_03"]
count_unique_01.columns = ['crid_encr', "phn_count_unique_01"]
count_unique_1w.columns = ['crid_encr', "phn_count_unique_1w"]

tmobile = tmobile.merge(count_unique_06, on = "crid_encr", how = "left")
print(tmobile.shape)
tmobile = tmobile.merge(count_unique_03, on = "crid_encr", how = "left")
print(tmobile.shape)
tmobile = tmobile.merge(count_unique_01, on = "crid_encr", how = "left")
print(tmobile.shape)
tmobile = tmobile.merge(count_unique_1w, on = "crid_encr", how = "left")
print(tmobile.shape)

(62414, 45)
(62414, 46)
(62414, 47)
(62414, 48)


In [36]:
temp = sbfe_for_tmobile[(sbfe_for_tmobile["credit_flag_cutoff_date_fmt"] - sbfe_for_tmobile["inquiry_date_fmt"])/np.timedelta64(1, 'M') <= 6].copy()
count_unique_06 = temp.groupby(by = ['crid_encr'])["taxidnumber"].nunique().reset_index()
print(count_unique_06.shape)

temp = sbfe_for_tmobile[(sbfe_for_tmobile["credit_flag_cutoff_date_fmt"] - sbfe_for_tmobile["inquiry_date_fmt"])/np.timedelta64(1, 'M') <= 3].copy()
count_unique_03 = temp.groupby(by = ['crid_encr'])["taxidnumber"].nunique().reset_index()
print(count_unique_03.shape)

temp = sbfe_for_tmobile[(sbfe_for_tmobile["credit_flag_cutoff_date_fmt"] - sbfe_for_tmobile["inquiry_date_fmt"])/np.timedelta64(1, 'M') <= 1].copy()
count_unique_01 = temp.groupby(by = ['crid_encr'])["taxidnumber"].nunique().reset_index()
print(count_unique_01.shape)

temp = sbfe_for_tmobile[(sbfe_for_tmobile["credit_flag_cutoff_date_fmt"] - sbfe_for_tmobile["inquiry_date_fmt"])/np.timedelta64(1, 'W') <= 1].copy()
count_unique_1w = temp.groupby(by = ['crid_encr'])["taxidnumber"].nunique().reset_index()
print(count_unique_1w.shape)

(12647, 2)
(9429, 2)
(5326, 2)
(2031, 2)


In [37]:
count_unique_06.columns = ['crid_encr', "taxid_count_unique_06"]
count_unique_03.columns = ['crid_encr', "taxid_count_unique_03"]
count_unique_01.columns = ['crid_encr', "taxid_count_unique_01"]
count_unique_1w.columns = ['crid_encr', "taxid_count_unique_1w"]

tmobile = tmobile.merge(count_unique_06, on = "crid_encr", how = "left")
print(tmobile.shape)
tmobile = tmobile.merge(count_unique_03, on = "crid_encr", how = "left")
print(tmobile.shape)
tmobile = tmobile.merge(count_unique_01, on = "crid_encr", how = "left")
print(tmobile.shape)
tmobile = tmobile.merge(count_unique_1w, on = "crid_encr", how = "left")
print(tmobile.shape)

(62414, 49)
(62414, 50)
(62414, 51)
(62414, 52)


In [38]:
tmobile['phn_count_unique_06'] = np.where(tmobile["phn_count_unique_06"].isnull(), 0,   tmobile["phn_count_unique_06"])
tmobile['phn_count_unique_03'] = np.where(tmobile["phn_count_unique_03"].isnull(), 0,   tmobile["phn_count_unique_03"])
tmobile['phn_count_unique_01'] = np.where(tmobile["phn_count_unique_01"].isnull(), 0,   tmobile["phn_count_unique_01"])
tmobile['phn_count_unique_1w'] = np.where(tmobile["phn_count_unique_1w"].isnull(), 0,   tmobile["phn_count_unique_1w"])

tmobile['taxid_count_unique_06'] = np.where(tmobile["taxid_count_unique_06"].isnull(), 0,   tmobile["taxid_count_unique_06"])
tmobile['taxid_count_unique_03'] = np.where(tmobile["taxid_count_unique_03"].isnull(), 0,   tmobile["taxid_count_unique_03"])
tmobile['taxid_count_unique_01'] = np.where(tmobile["taxid_count_unique_01"].isnull(), 0,   tmobile["taxid_count_unique_01"])
tmobile['taxid_count_unique_1w'] = np.where(tmobile["taxid_count_unique_1w"].isnull(), 0,   tmobile["taxid_count_unique_1w"])

In [39]:
tmobile["fraud_ind"] = tmobile["fraud_ind"].astype(float)

In [40]:
fmt = make_format(cuts = [-np.inf, 0, 1, 2, 3, 4, 5, 10, 20, 30, 40, 50, np.inf], exceptions = [-1])
result = dict()
keep = ['phn_count_unique_06', 'phn_count_unique_03', 'phn_count_unique_01', 'phn_count_unique_1w', 'taxid_count_unique_06', 'taxid_count_unique_03', 'taxid_count_unique_01', 'taxid_count_unique_1w']
for i, a in enumerate(keep):
    result[i] = bivariate(tmobile[a], tmobile["fraud_ind"], format = fmt)

wb = TableWriter(filename = "./_temp/tmobile.xlsx", options={'nan_inf_to_errors': True}, overwrite = True)
for k in result.keys():
    wb.write_table(
        result[k],
        sheetname="biv",
        conditional_fmt_cols=[3],
    )
wb.close()

#### Chase

In [41]:
chase = pd.read_adls("Analytics/RnD Projects/Product RnD/Business/LexisNexis 9999 (SBFE Inquiry POC)/Performance/Fraud Sample/Chase_11449/chase_stacked_final.parquet")
print(chase.shape)

(640958, 36)


In [42]:
chase["credit_flag_cutoff_date_fmt"] = pd.to_datetime(chase["app_date"], format='%d%b%Y')

In [43]:
chase.columns

Index(['transaction_id', 'cust_first_nm', 'cust_mid_init_tx', 'cust_last_nm',
       'govt_issu_id_nb', 'line_1_ad', 'line_2_ad', 'city_nm', 'state_prov_cd',
       'pst_area_7_cd', 'pst_area_cd', 'channel', 'bus_name', 'app_date',
       'date_of_birth', 'decision', 'bizidscore', 'bd_score', 'bd_score1',
       'bd_score2', 'bd_score3', 'bus_addr1', 'bus_addr2', 'bus_city',
       'bus_state', 'bus_zip', 'bus_tax_id', 'bus_phone', 'bus_type',
       'home_phone', 'bad', 'seq', 'account', 'date', 'LN_booked', 'count',
       'credit_flag_cutoff_date_fmt'],
      dtype='object')

In [44]:
# Compile Full Address: np.where(stripe["bus_streetaddress2"] != "", ", " + stripe["bus_streetaddress2"], "") + 
chase["full_addr"] = chase["bus_addr1"] + \
    np.where(chase["bus_city"] != "", ", " + chase["bus_city"], "") + \
    np.where(chase["bus_state"] != "", ", " + chase["bus_state"], "") + \
        np.where(chase["bus_zip"] != "", ", " + chase["bus_zip"].str.slice(0, 5), "")
chase["full_addr_clean"] = chase['full_addr'].str.replace('.', '')
chase["full_addr_clean"] = chase['full_addr_clean'].str.replace(',', '')
chase["full_addr_clean"] = chase['full_addr_clean'].str.replace('\W\s', '')
chase["full_addr_clean"] = chase["full_addr_clean"].str.lower()
chase["full_addr_clean"]

/tmp/ipykernel_25390/3843543105.py:6: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  chase["full_addr_clean"] = chase['full_addr'].str.replace('.', '')
/tmp/ipykernel_25390/3843543105.py:8: FutureWarning: The default value of regex will change from True to False in a future version.
  chase["full_addr_clean"] = chase['full_addr_clean'].str.replace('\W\s', '')


499427                 2869 sw 33rd ct unit 1 miami fl 33133
837612              821 michigan ave apt 3 evanston il 60202
1018748             8204 n lamar blvd ste a4 austin tx 78753
82631                      20558 noland woods ct sterling va
37307                              23 n midland ave nyack ny
                                 ...                        
156867                 2222 s dobson rd apt 1089 chandler az
140946                          1018 n morrison st kokomo in
62551                  597 westport ave unit a420 norwalk ct
1225640    17070 collins ave ste 255 sunny isles beach fl...
49725                 8324 seven falls st north las vegas nv
Name: full_addr_clean, Length: 640958, dtype: object

In [45]:
print(chase[chase["full_addr_clean"]==""].shape)
print(chase[chase["full_addr_clean"].isnull()].shape)

(0, 39)
(0, 39)


In [46]:
sbfe_for_chase = sbfe.merge(chase, on = "full_addr_clean")
print(sbfe_for_chase.shape)
sbfe_for_chase = sbfe_for_chase[sbfe_for_chase["inquiry_date_fmt"] < sbfe_for_chase["credit_flag_cutoff_date_fmt"]]
print(sbfe_for_chase.shape)

(2658103, 44)
(1921750, 44)


In [47]:
temp = sbfe_for_chase[(sbfe_for_chase["credit_flag_cutoff_date_fmt"] - sbfe_for_chase["inquiry_date_fmt"])/np.timedelta64(1, 'M') <= 6].copy()
count_unique_06 = temp.groupby(by = ['account'])["businessphone"].nunique().reset_index()
print(count_unique_06.shape)

temp = sbfe_for_chase[(sbfe_for_chase["credit_flag_cutoff_date_fmt"] - sbfe_for_chase["inquiry_date_fmt"])/np.timedelta64(1, 'M') <= 3].copy()
count_unique_03 = temp.groupby(by = ['account'])["businessphone"].nunique().reset_index()
print(count_unique_03.shape)

temp = sbfe_for_chase[(sbfe_for_chase["credit_flag_cutoff_date_fmt"] - sbfe_for_chase["inquiry_date_fmt"])/np.timedelta64(1, 'M') <= 1].copy()
count_unique_01 = temp.groupby(by = ['account'])["businessphone"].nunique().reset_index()
print(count_unique_01.shape)

temp = sbfe_for_chase[(sbfe_for_chase["credit_flag_cutoff_date_fmt"] - sbfe_for_chase["inquiry_date_fmt"])/np.timedelta64(1, 'W') <= 1].copy()
count_unique_1w = temp.groupby(by = ['account'])["businessphone"].nunique().reset_index()
print(count_unique_1w.shape)

(80884, 2)
(59972, 2)
(34910, 2)
(15193, 2)


In [48]:
count_unique_06.columns = ['account', "phn_count_unique_06"]
count_unique_03.columns = ['account', "phn_count_unique_03"]
count_unique_01.columns = ['account', "phn_count_unique_01"]
count_unique_1w.columns = ['account', "phn_count_unique_1w"]

chase = chase.merge(count_unique_06, on = "account", how = "left")
print(chase.shape)
chase = chase.merge(count_unique_03, on = "account", how = "left")
print(chase.shape)
chase = chase.merge(count_unique_01, on = "account", how = "left")
print(chase.shape)
chase = chase.merge(count_unique_1w, on = "account", how = "left")
print(chase.shape)

(640958, 40)
(640958, 41)
(640958, 42)
(640958, 43)


In [49]:
temp = sbfe_for_chase[(sbfe_for_chase["credit_flag_cutoff_date_fmt"] - sbfe_for_chase["inquiry_date_fmt"])/np.timedelta64(1, 'M') <= 6].copy()
count_unique_06 = temp.groupby(by = ['account'])["taxidnumber"].nunique().reset_index()
print(count_unique_06.shape)

temp = sbfe_for_chase[(sbfe_for_chase["credit_flag_cutoff_date_fmt"] - sbfe_for_chase["inquiry_date_fmt"])/np.timedelta64(1, 'M') <= 3].copy()
count_unique_03 = temp.groupby(by = ['account'])["taxidnumber"].nunique().reset_index()
print(count_unique_03.shape)

temp = sbfe_for_chase[(sbfe_for_chase["credit_flag_cutoff_date_fmt"] - sbfe_for_chase["inquiry_date_fmt"])/np.timedelta64(1, 'M') <= 1].copy()
count_unique_01 = temp.groupby(by = ['account'])["taxidnumber"].nunique().reset_index()
print(count_unique_01.shape)

temp = sbfe_for_chase[(sbfe_for_chase["credit_flag_cutoff_date_fmt"] - sbfe_for_chase["inquiry_date_fmt"])/np.timedelta64(1, 'W') <= 1].copy()
count_unique_1w = temp.groupby(by = ['account'])["taxidnumber"].nunique().reset_index()
print(count_unique_1w.shape)

(80884, 2)
(59972, 2)
(34910, 2)
(15193, 2)


In [50]:
count_unique_06.columns = ['account', "taxid_count_unique_06"]
count_unique_03.columns = ['account', "taxid_count_unique_03"]
count_unique_01.columns = ['account', "taxid_count_unique_01"]
count_unique_1w.columns = ['account', "taxid_count_unique_1w"]

chase = chase.merge(count_unique_06, on = "account", how = "left")
print(chase.shape)
chase = chase.merge(count_unique_03, on = "account", how = "left")
print(chase.shape)
chase = chase.merge(count_unique_01, on = "account", how = "left")
print(chase.shape)
chase = chase.merge(count_unique_1w, on = "account", how = "left")
print(chase.shape)

(640958, 44)
(640958, 45)
(640958, 46)
(640958, 47)


In [51]:
chase['phn_count_unique_06'] = np.where(chase["phn_count_unique_06"].isnull(), 0, chase["phn_count_unique_06"])
chase['phn_count_unique_03'] = np.where(chase["phn_count_unique_03"].isnull(), 0, chase["phn_count_unique_03"])
chase['phn_count_unique_01'] = np.where(chase["phn_count_unique_01"].isnull(), 0, chase["phn_count_unique_01"])
chase['phn_count_unique_1w'] = np.where(chase["phn_count_unique_1w"].isnull(), 0, chase["phn_count_unique_1w"])

chase['taxid_count_unique_06'] = np.where(chase["taxid_count_unique_06"].isnull(), 0, chase["taxid_count_unique_06"])
chase['taxid_count_unique_03'] = np.where(chase["taxid_count_unique_03"].isnull(), 0, chase["taxid_count_unique_03"])
chase['taxid_count_unique_01'] = np.where(chase["taxid_count_unique_01"].isnull(), 0, chase["taxid_count_unique_01"])
chase['taxid_count_unique_1w'] = np.where(chase["taxid_count_unique_1w"].isnull(), 0, chase["taxid_count_unique_1w"])

In [53]:
chase["fraud_ind"] = chase["bad"].astype(float)

In [54]:
fmt = make_format(cuts = [-np.inf, 0, 1, 2, 3, 4, 5, 10, 20, 30, 40, 50, np.inf], exceptions = [-1])
result = dict()
keep = ['phn_count_unique_06', 'phn_count_unique_03', 'phn_count_unique_01', 'phn_count_unique_1w', 'taxid_count_unique_06', 'taxid_count_unique_03', 'taxid_count_unique_01', 'taxid_count_unique_1w']
for i, a in enumerate(keep):
    result[i] = bivariate(chase[a].astype(float), chase["fraud_ind"], format = fmt)

wb = TableWriter(filename = "./_temp/chase.xlsx", options={'nan_inf_to_errors': True}, overwrite = True)
for k in result.keys():
    wb.write_table(
        result[k],
        sheetname="biv",
        conditional_fmt_cols=[3],
    )
wb.close()

#### Fleector

In [55]:
fleector = pd.read_adls("Analytics/RnD Projects/Product RnD/Business/LexisNexis 9999 (SBFE Inquiry POC)/Performance/Fraud Sample/fleector_11124/fleector_sample_to_use.parquet")
print(fleector.shape)

(83704, 92)


In [56]:
fleector["credit_flag_cutoff_date_fmt"] = pd.to_datetime(fleector["date_created"], format='%Y-%m-%d')

In [57]:
freq(fleector["credit_flag_cutoff_date_fmt"].astype(str).str.slice(0, 7))

,Count,Pct,Cuml Count,Cuml Pct
credit_flag_cutoff_date_fmt,,,,
2022-01,15827.0,0.189083,15827.0,0.189083
2022-02,15512.0,0.185320,31339.0,0.374403
2022-03,19353.0,0.231208,50692.0,0.605610
2022-04,16822.0,0.200970,67514.0,0.806580
2022-05,16185.0,0.193360,83699.0,0.999940
2022-06,5.0,0.000060,83704.0,1.000000


In [58]:
# Compile Full Address: np.where(stripe["bus_streetaddress2"] != "", ", " + stripe["bus_streetaddress2"], "") + 
fleector["full_addr"] = fleector["cust_address1"] + \
    np.where(fleector["cust_city"] != "", ", " + fleector["cust_city"], "") + \
    np.where(fleector["cust_state"] != "", ", " + fleector["cust_state"], "") + \
        np.where(fleector["cust_zip"] != "", ", " + fleector["cust_zip"].str.slice(0, 5), "")
fleector["full_addr_clean"] = fleector['full_addr'].str.replace('.', '')
fleector["full_addr_clean"] = fleector['full_addr_clean'].str.replace(',', '')
fleector["full_addr_clean"] = fleector['full_addr_clean'].str.replace('\W\s', '')
fleector["full_addr_clean"] = fleector["full_addr_clean"].str.lower()
fleector["full_addr_clean"]

/tmp/ipykernel_25390/2826320773.py:6: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  fleector["full_addr_clean"] = fleector['full_addr'].str.replace('.', '')
/tmp/ipykernel_25390/2826320773.py:8: FutureWarning: The default value of regex will change from True to False in a future version.
  fleector["full_addr_clean"] = fleector['full_addr_clean'].str.replace('\W\s', '')


6875                  637 sackman st brooklyn ny 11212
5628     413timberline ridge ct winston salem nc 27106
19723          170bentley place way covington ga 30016
4981                99 horace lane walterboro sc 29488
17116           3828 highlands pkwy se smyrna ga 30082
                             ...                      
49448           10387 tiago lake dr escondido ca 92029
705          645 henderson drive cartersville ga 30120
79073             98 whitethorne ave columbus oh 43223
61964               107 floridana road debary fl 32713
54909           4207 youngstown dr greensboro nc 27405
Name: full_addr_clean, Length: 83704, dtype: object

In [59]:
print(fleector[fleector["full_addr_clean"]==""].shape)
print(fleector[fleector["full_addr_clean"].isnull()].shape)

(0, 95)
(0, 95)


In [60]:
fleector.seq.is_unique

True

In [62]:
fleector.shape

(83704, 95)

In [61]:
sbfe_for_fleector = sbfe.merge(fleector, on = "full_addr_clean")
print(sbfe_for_fleector.shape)
sbfe_for_fleector = sbfe_for_fleector[sbfe_for_fleector["inquiry_date_fmt"] < sbfe_for_fleector["credit_flag_cutoff_date_fmt"]]
print(sbfe_for_fleector.shape)

(176804, 100)
(89741, 100)


In [64]:
temp = sbfe_for_fleector[(sbfe_for_fleector["credit_flag_cutoff_date_fmt"] - sbfe_for_fleector["inquiry_date_fmt"])/np.timedelta64(1, 'M') <= 6].copy()
count_unique_06 = temp.groupby(by = ['seq'])["businessphone"].nunique().reset_index()
print(count_unique_06.shape)

temp = sbfe_for_fleector[(sbfe_for_fleector["credit_flag_cutoff_date_fmt"] - sbfe_for_fleector["inquiry_date_fmt"])/np.timedelta64(1, 'M') <= 3].copy()
count_unique_03 = temp.groupby(by = ['seq'])["businessphone"].nunique().reset_index()
print(count_unique_03.shape)

temp = sbfe_for_fleector[(sbfe_for_fleector["credit_flag_cutoff_date_fmt"] - sbfe_for_fleector["inquiry_date_fmt"])/np.timedelta64(1, 'M') <= 1].copy()
count_unique_01 = temp.groupby(by = ['seq'])["businessphone"].nunique().reset_index()
print(count_unique_01.shape)

temp = sbfe_for_fleector[(sbfe_for_fleector["credit_flag_cutoff_date_fmt"] - sbfe_for_fleector["inquiry_date_fmt"])/np.timedelta64(1, 'W') <= 1].copy()
count_unique_1w = temp.groupby(by = ['seq'])["businessphone"].nunique().reset_index()
print(count_unique_1w.shape)

(12875, 2)
(9748, 2)
(6064, 2)
(2471, 2)


In [65]:
count_unique_06.columns = ['seq', "phn_count_unique_06"]
count_unique_03.columns = ['seq', "phn_count_unique_03"]
count_unique_01.columns = ['seq', "phn_count_unique_01"]
count_unique_1w.columns = ['seq', "phn_count_unique_1w"]

fleector = fleector.merge(count_unique_06, on = "seq", how = "left")
print(fleector.shape)
fleector = fleector.merge(count_unique_03, on = "seq", how = "left")
print(fleector.shape)
fleector = fleector.merge(count_unique_01, on = "seq", how = "left")
print(fleector.shape)
fleector = fleector.merge(count_unique_1w, on = "seq", how = "left")
print(fleector.shape)

(83704, 96)
(83704, 97)
(83704, 98)
(83704, 99)


In [66]:
temp = sbfe_for_fleector[(sbfe_for_fleector["credit_flag_cutoff_date_fmt"] - sbfe_for_fleector["inquiry_date_fmt"])/np.timedelta64(1, 'M') <= 6].copy()
count_unique_06 = temp.groupby(by = ['seq'])["taxidnumber"].nunique().reset_index()
print(count_unique_06.shape)

temp = sbfe_for_fleector[(sbfe_for_fleector["credit_flag_cutoff_date_fmt"] - sbfe_for_fleector["inquiry_date_fmt"])/np.timedelta64(1, 'M') <= 3].copy()
count_unique_03 = temp.groupby(by = ['seq'])["taxidnumber"].nunique().reset_index()
print(count_unique_03.shape)

temp = sbfe_for_fleector[(sbfe_for_fleector["credit_flag_cutoff_date_fmt"] - sbfe_for_fleector["inquiry_date_fmt"])/np.timedelta64(1, 'M') <= 1].copy()
count_unique_01 = temp.groupby(by = ['seq'])["taxidnumber"].nunique().reset_index()
print(count_unique_01.shape)

temp = sbfe_for_fleector[(sbfe_for_fleector["credit_flag_cutoff_date_fmt"] - sbfe_for_fleector["inquiry_date_fmt"])/np.timedelta64(1, 'W') <= 1].copy()
count_unique_1w = temp.groupby(by = ['seq'])["taxidnumber"].nunique().reset_index()
print(count_unique_1w.shape)

(12875, 2)
(9748, 2)
(6064, 2)
(2471, 2)


In [67]:
count_unique_06.columns = ['seq', "taxid_count_unique_06"]
count_unique_03.columns = ['seq', "taxid_count_unique_03"]
count_unique_01.columns = ['seq', "taxid_count_unique_01"]
count_unique_1w.columns = ['seq', "taxid_count_unique_1w"]

fleector = fleector.merge(count_unique_06, on = "seq", how = "left")
print(fleector.shape)
fleector = fleector.merge(count_unique_03, on = "seq", how = "left")
print(fleector.shape)
fleector = fleector.merge(count_unique_01, on = "seq", how = "left")
print(fleector.shape)
fleector = fleector.merge(count_unique_1w, on = "seq", how = "left")
print(fleector.shape)

(83704, 100)
(83704, 101)
(83704, 102)
(83704, 103)


In [68]:
fleector['phn_count_unique_06'] = np.where(fleector["phn_count_unique_06"].isnull(), 0, fleector["phn_count_unique_06"])
fleector['phn_count_unique_03'] = np.where(fleector["phn_count_unique_03"].isnull(), 0, fleector["phn_count_unique_03"])
fleector['phn_count_unique_01'] = np.where(fleector["phn_count_unique_01"].isnull(), 0, fleector["phn_count_unique_01"])
fleector['phn_count_unique_1w'] = np.where(fleector["phn_count_unique_1w"].isnull(), 0, fleector["phn_count_unique_1w"])

fleector['taxid_count_unique_06'] = np.where(fleector["taxid_count_unique_06"].isnull(), 0, fleector["taxid_count_unique_06"])
fleector['taxid_count_unique_03'] = np.where(fleector["taxid_count_unique_03"].isnull(), 0, fleector["taxid_count_unique_03"])
fleector['taxid_count_unique_01'] = np.where(fleector["taxid_count_unique_01"].isnull(), 0, fleector["taxid_count_unique_01"])
fleector['taxid_count_unique_1w'] = np.where(fleector["taxid_count_unique_1w"].isnull(), 0, fleector["taxid_count_unique_1w"])

In [69]:
fleector["fraud_ind"] = fleector["final_fraud_flag"].astype(float)

In [70]:
fmt = make_format(cuts = [-np.inf, 0, 1, 2, 3, 4, 5, 10, 20, 30, 40, 50, np.inf], exceptions = [-1])
result = dict()
keep = ['phn_count_unique_06', 'phn_count_unique_03', 'phn_count_unique_01', 'phn_count_unique_1w', 
        'taxid_count_unique_06', 'taxid_count_unique_03', 'taxid_count_unique_01', 'taxid_count_unique_1w']
for i, a in enumerate(keep):
    result[i] = bivariate(fleector[a].astype(float), fleector["fraud_ind"], format = fmt, groups= fleector.acct_flag)

wb = TableWriter(filename = "./_temp/fleector.xlsx", options={'nan_inf_to_errors': True}, overwrite = True)
for k in result.keys():
    wb.write_table(
        result[k],
        sheetname="biv",
        conditional_fmt_cols=[3],
    )
wb.close()

#### Other

In [25]:
statename_to_abbr = {
    # Other
    'District of Columbia': 'DC',

    # States
    'Alabama': 'AL',
    'Montana': 'MT',
    'Alaska': 'AK',
    'Nebraska': 'NE',
    'Arizona': 'AZ',
    'Nevada': 'NV',
    'Arkansas': 'AR',
    'New Hampshire': 'NH',
    'California': 'CA',
    'New Jersey': 'NJ',
    'Colorado': 'CO',
    'New Mexico': 'NM',
    'Connecticut': 'CT',
    'New York': 'NY',
    'Delaware': 'DE',
    'North Carolina': 'NC',
    'Florida': 'FL',
    'North Dakota': 'ND',
    'Georgia': 'GA',
    'Ohio': 'OH',
    'Hawaii': 'HI',
    'Oklahoma': 'OK',
    'Idaho': 'ID',
    'Oregon': 'OR',
    'Illinois': 'IL',
    'Pennsylvania': 'PA',
    'Indiana': 'IN',
    'Rhode Island': 'RI',
    'Iowa': 'IA',
    'South Carolina': 'SC',
    'Kansas': 'KS',
    'South Dakota': 'SD',
    'Kentucky': 'KY',
    'Tennessee': 'TN',
    'Louisiana': 'LA',
    'Texas': 'TX',
    'Maine': 'ME',
    'Utah': 'UT',
    'Maryland': 'MD',
    'Vermont': 'VT',
    'Massachusetts': 'MA',
    'Virginia': 'VA',
    'Michigan': 'MI',
    'Washington': 'WA',
    'Minnesota': 'MN',
    'West Virginia': 'WV',
    'Mississippi': 'MS',
    'Wisconsin': 'WI',
    'Missouri': 'MO',
    'Wyoming': 'WY',
    "Puerto Rico" : "PR",
"Virgin Islands" : "VI"
}